In [ ]:
!pip install opencv-python-headless tensorflow


In [ ]:
import torch
import torchvision
import cv2
from torchvision.models.detection import SSD300_VGG16_Weights

# Load pretrained SSD model
weights = SSD300_VGG16_Weights.DEFAULT
model = torchvision.models.detection.ssd300_vgg16(weights=weights)
model.eval()

# COCO class labels from weights
labels = weights.meta["categories"]

print("SSD model loaded!")

Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to /root/.cache/torch/hub/checkpoints/ssd300_vgg16_coco-b556d3b4.pth


100%|██████████| 136M/136M [00:01<00:00, 108MB/s]


SSD model loaded!


In [ ]:
from google.colab import files
uploaded = files.upload()

video_path = list(uploaded.keys())[0]
print("Uploaded video:", video_path)


Saving EV20250426-144029-000346F.mp4 to EV20250426-144029-000346F.mp4
Uploaded video: EV20250426-144029-000346F.mp4


In [ ]:
import numpy as np

cap = cv2.VideoCapture(video_path)

# Output video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(
    "annotated_output.mp4",
    fourcc,
    30.0,
    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
     int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
)

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to tensor
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_tensor = torch.tensor(img_rgb / 255.0).permute(2, 0, 1).float()

    # Run model
    with torch.no_grad():
        result = model([img_tensor])[0]

    # Draw detections
    for box, score, cls in zip(result['boxes'], result['scores'], result['labels']):
        if score > 0.5:  # confidence threshold
            x1, y1, x2, y2 = box.int().tolist()
            label = labels[int(cls)]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {score:.2f}",
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 255, 0), 2)

    out.write(frame)
    frame_count += 1

print("Processing complete! Frames processed:", frame_count)

cap.release()
out.release()
#24 min

Processing complete! Frames processed: 945


In [ ]:
from google.colab import files
files.download("annotated_output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>